In [ ]:
#install all packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import plotly.express as px

In [ ]:
%pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 756.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.1 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=0ecdb8e6b0a6035b4c868c1b7b48a25f087b42971a738436fc82f51c0dd00024
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE #TSNE is t-stochastic neighbor embeddings
from umap.umap_ import UMAP
from mpl_toolkits.mplot3d import axes3d

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Intro - Strategic Store Expansion Using Demographic Data

In this project, we aim to uncover insights into the relationship between household debt to income ratio and the strategic decisions of store openings by Target.  While we don't have access to the performance data of existing stores, we can use demographic and economic information to identify regions with high potential for new store success based on regions that already have a Target store. By analyzing demographic and economic characteristics of different counties we can recommend prime locations for Target's expansion.

Our key question is: Where should we open the next Target?

We will be using 4 datasets for this project:

1.   targets.csv - This dataset includes a record for Target locations currently in operation as of April 2017
2.   debt_county.csv - This dataset includes a record of the household debt to income ration since 1999
3.    pop_age_sex.csv - This dataset includes a record of population by age and sex per county
4.    race_data.csv - This dataset includes a record of race per county

Let's begin the cleaning process for target.csv

## Target Data Inspection

In [ ]:
raw_target_data = pd.read_csv('/content/drive/MyDrive/Brainstation/Target Capstone /datasets/target.csv', encoding='latin1')
raw_target_data.head()

Here we can see the first 5 rows in our dataset and understand the types of data we will be working with, we see a combination of date, continues and categorical data. Let's check the shape of this data to understand how large the dataset is.

In [ ]:
raw_target_data.shape

Here we see that there are 1829 rows/stores and 47 features. Let's check for duplicate values to ensure every row is unique.

In [ ]:
raw_target_data.duplicated().sum()

We see that there are no duplicated, but to be sure let's ensure there are no duplicate addresses. This ensures there are no duplicate stores even if the complete row is not completely the same.


In [ ]:
raw_target_data.duplicated(subset=['Address.AddressLine1']).sum()

Here we see again that there are no duplicates in terms of addresses.

Let's analyze the number of null values in our data set

In [ ]:
raw_target_data.isnull().sum()

We've identified null values in several columns, including 'Address.AddressLine2' (1771 nulls), 'SubTypeDescription' (1522 nulls), 'LocationMilestones.LastRemodelDate' (395 nulls), 'Capabilities' (12 nulls), and 'Market' (91 nulls). These nulls may reflect non-remodeled locations or a lack of additional capabilities. Further analysis is needed to determine the most appropriate handling of these null values.

Let's now assess which columns are impactful to our analysis of answering our key question: Where should we open a new Target location?

Let's list out all of the columns in the dataset

### Evaluating Column Relevance

In [ ]:
raw_target_data.columns

In this section, we observe various fields related to store opening times. While these variables are noteworthy, they are not currently pertinent to our analysis and will therefore be excluded. The focus will shift to other relevant data, which I plan to explore in more detail later in the notebook.

The fields concerning store opening times will now be removed from the current dataset.

In [ ]:
# List of columns to drop
columns_to_drop = [
    'BeginTime.MF', 'Is24Hours.MF', 'IsOpen.MF', 'Summary.MF',
    'ThruTime.MF', 'BeginTime.Sat', 'Is24Hours.Sat', 'IsOpen.Sat',
    'Summary.Sat', 'ThruTime.Sat', 'BeginTime.Sun', 'Is24Hours.Sun',
    'IsOpen.Sun', 'Summary.Sun', 'ThruTime.Sun',
    'TimeZone.TimeZoneCode', 'TimeZone.TimeZoneDescription',
    'TimeZone.TimeZoneOffset.OffsetCode', 'OperatingHours..timeFormat',
    'TimeZone.TimeZoneOffset.OffsetHours'
]

# Drop the columns
raw_target_data_drop = raw_target_data.drop(columns=columns_to_drop)

#check for relevant columns
raw_target_data_drop.columns

With the removal of columns pertaining to store operating hours complete, our next step is to assess the remaining columns for their relevance to our analysis. Immediately, it becomes apparent that some features, such as phone numbers, fax numbers, and the 'recognition of daylight savings' status, are not pertinent to our objectives. As such, we will proceed to discard these irrelevant features.

In [ ]:
# List of columns to drop
columns_to_drop2 = ['PhoneNumber', 'FaxNumber', 'IsDaylightSavingsTimeRecognized',
                    'Address.IntersectionDescription','Address.CountryName','AlternateIdentifier.ID','ID']

# Drop the columns
raw_target_data_drop.drop(columns=columns_to_drop2, inplace=True)

In [ ]:


#Check Columns
raw_target_data_drop.columns


Now that we're down to 24 columns, at first glance, they all seem relevant. However, to ensure a thorough analysis, we will examine each column individually. This detailed exploration will help us understand the relevance of each column in the context of our primary question: Where should we open the next Target location?

### Column Cleaning

Let's inspect each column to better understand what they contain let's start with name

In [ ]:
raw_target_data['Name']

Looks like the name column contains the names of stores, this is somewhat redundant since we have addresses to identify the store.

Let's evaluate the Address Line 2 column

In [ ]:
#explore address.line2
raw_target_data_drop['Address.AddressLine2'].unique()

Here we see the Adress Line 2 data which primarily contains unit numbers. Given that we already have the main address details in a separate field, and their are many nulls due to many locations not having a line 2 in their address -- 'Address Line 2' appears to be of limited relevance to our analysis.

We will drop this feature

In [ ]:
#drop AddressLine2
raw_target_data_drop1 = raw_target_data_drop.drop('Address.AddressLine2', axis=1)

#check drop
raw_target_data_drop1.columns

Now we can explore the subtype description field

In [ ]:
#explore subtype description

raw_target_data_drop1['SubTypeDescription'].unique()

This column offers potentially valuable insights, as it categorizes the type of Target store, such as 'SuperTarget', 'TargetExpress', or 'City'. To address the issue of missing (NaN) values, one approach could be to enhance the dataset through web scraping. However, for the sake of simplicity and immediate analysis, I will initially assume that these NaN entries represent standard Target stores. Accordingly, I'll map all NaN values in this column to 'Regular', enabling a more streamlined and comprehensive analysis of store types.

In [ ]:
#replace all null values with 'Regular'
raw_target_data_drop1['SubTypeDescription'].fillna('Regular',inplace = True)


In [ ]:
#check for no null values
raw_target_data_drop1['SubTypeDescription'].isnull().sum()

In [ ]:
#check for unique values
raw_target_data_drop1['SubTypeDescription'].unique()

We now see that there are no nulls in the SubTypeDescription column and all NaN have been replaced with 'Regular'

Now it's time to explore the X.locale column

In [ ]:
#explore X.Locale
raw_target_data_drop1['X.locale'].unique()

This column only contains 'en-US' meaning all locations are the same we can drop this column as it is redundant.

In [ ]:
#drop x.locale
raw_target_data_drop2 = raw_target_data_drop1.drop('X.locale', axis=1)

#check drop
raw_target_data_drop2.columns

Now let's evaluate the LastRemodeled field.
The 'LastRemodeled' field is potentially crucial, as it may indicate a store's success through Target's reinvestment, offering indirect insights into store performance in the absence of direct sales data.

In [ ]:
#explore last remodeled field
raw_target_data_drop2['LocationMilestones.LastRemodelDate'].isna().sum()

We see that there are 395 nulls to handle. Because this field's nulls could mean that the Target wasnt remodeled we will create a new field with a binary indicator. Set it to 1 if 'LocationMilestones.LastRemodelDate' is not null and 0 otherwise.

In [ ]:

# Create a binary indicator column
raw_target_data_drop2['Remodeled'] = pd.notna(raw_target_data['LocationMilestones.LastRemodelDate']).astype(int)

#check new column
raw_target_data_drop2[['LocationMilestones.LastRemodelDate','Remodeled']]


We can now see the Remodeled column ther 0's where there are NaNs and 1's where there are dates.



Let's convert the open date and last remodeled date column to pandas datetime in order to be able to use it for analysis later.

In [ ]:
# Convert the columns to datetime format
raw_target_data_drop2['LocationMilestones.OpenDate'] = pd.to_datetime(raw_target_data_drop2['LocationMilestones.OpenDate'])
raw_target_data_drop2['LocationMilestones.LastRemodelDate'] = pd.to_datetime(raw_target_data_drop2['LocationMilestones.LastRemodelDate'])


In [ ]:
raw_target_data_drop2[['LocationMilestones.OpenDate','LocationMilestones.LastRemodelDate']]

Now let's explore the Market Column

In [ ]:
raw_target_data_drop2['Market'].unique()

I was unable to find any documentation on what this field means so I will drop it.

In [ ]:
#drop Market column
raw_target_data_drop3 = raw_target_data_drop2.drop('Market', axis = 1)

#check for clean data
raw_target_data_drop3.columns

Now let's explore 'All Capability'

In [ ]:
raw_target_data_drop3['AllCapability']

We see that this field contains lists of additional features that a Target has such as a CVS, Cafe, Bakery etc. This could be useful for analysis. In order to process this I will create binary indicators for each unique value.

First let's handle the null values by filling them with an empty list

In [ ]:


# Replace NaN values with an empty list
raw_target_data_drop3['AllCapability'].fillna('[]', inplace=True)



Now let's remove unwanted characters such as commas and brackets then use a string method that splits each entry in the 'AllCapability' column into a list, using the comma as a delimiter.

In [ ]:
# Remove unwanted characters from the string and split the values into lists
raw_target_data_drop3['AllCapability'] = raw_target_data_drop3['AllCapability'].str.replace("[\[\]']", '', regex=True)
raw_target_data_drop3['AllCapability'] = raw_target_data_drop3['AllCapability'].str.split(', ')



In [ ]:
raw_target_data_drop3['AllCapability']

The 'AllCapability' column now comprises a series of capabilities, but they are not yet in a Python list format suitable for analysis. To address this, we will create a new column designed to store these capabilities as proper Python lists, facilitating further manipulation and analysis.

In [ ]:
# Assuming raw_target_data_drop3['AllCapability'] contains lists of capabilities
# Convert each list of capabilities into a string of comma-separated values
raw_target_data_drop3['AllCapability_str'] = raw_target_data_drop3['AllCapability'].apply(lambda x: ','.join(x) if isinstance(x, list) else '')




With the creation of a column containing properly formatted lists, we're now in a position to apply one-hot encoding to each capability. This can be achieved using the 'get_dummies' method, effectively transforming each unique capability into its own binary column.

In [ ]:
# Now use `str.get_dummies` which works on comma-separated strings to create one-hot encoding
capabilities_dummies = raw_target_data_drop3['AllCapability_str'].str.get_dummies(sep=',')

# Join the one-hot encoded DataFrame with the original DataFrame
raw_target_data_drop3 = raw_target_data_drop3.join(capabilities_dummies)

# You can drop the intermediate 'AllCapability_str' if it's no longer needed
raw_target_data_drop3.drop('AllCapability_str', axis=1, inplace=True)

With the one hot encoded features now added to the dataset we can drop the AllCapabilities column

In [ ]:

# Drop the original 'AllCapability' column
raw_target_data_drop3.drop('AllCapability', axis=1, inplace=True)

In [ ]:
raw_target_data_drop3.head()

In [ ]:
raw_target_data_drop3.columns

We now see each Capability hot-encoded as a binary values

### Statistical Analysis

Let's explore the statistical distribution and possible correlations in this dataset. We'll start with a heat map of all numerical columns.

In [ ]:
# Define column groups
groups = [
    ['Address.Latitude', 'Address.Longitude', 'Remodeled', 'Address.AddressLine1', 'Address.City', 'Address.County', 'Address.PostalCode'],
    ['Address.Latitude', 'Address.Longitude', 'Remodeled', 'Name'],
    ['Address.Latitude', 'Address.Longitude', 'Remodeled', 'Bakery', 'Beer', 'Cafe-Pizza', 'Café'],
    ['Address.Latitude', 'Address.Longitude', 'Remodeled', 'CVS pharmacy', 'MinuteClinic', 'Optical', 'Photo Lab'],
    ['Address.Latitude', 'Address.Longitude', 'Remodeled', 'Drive Up', 'Fresh Grocery', 'Starbucks']
]

#Loop through each group to compute and display correlation matrix
for index, group in enumerate(groups, 1):
    correlation_matrix = raw_target_data_drop3[group].corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title(f"Correlation Matrix for Group {index}")
    plt.show()


Upon examining the correlation matrix, we observe predominantly weak linear relationships between variables, hinting that they might largely function independently. This independence mitigates concerns about multicollinearity. However, these low correlations don't necessarily indicate an absence of relationships; non-linear interactions might be present but not reflected in the current metrics. While the initial impression suggests some variables might have lesser relevance due to their low correlation with the target, I will approach feature selection with a holistic view by exploring other types of relationships

###Histogram Analysis

Let's now plot histograms for each of the numeric columns

In [ ]:
numeric_raw = raw_target_data_drop3.select_dtypes(include=['number'])

for col in numeric_raw.columns:
  numeric_raw.loc[:,col].plot(kind='hist')
  plt.title(col)
  plt.show()

###Analysis of Histograms

**Latitude and Longitude:**
The distribution of the latitude and longitude values appears to be normal. This suggests a fairly even distribution of Target stores across the regions under consideration. Visualizing these coordinates on a map or plotting them in a scatter plot could provide more granular insights about store distribution relative to geographical regions.

**Remodeled Stores:**
The data indicates a higher number of stores that have undergone remodeling compared to those that haven't. This is an intriguing observation, as it suggests that Target is keen on reinvesting in existing locations. It would be valuable to explore what types of neighborhoods or areas Target is choosing to reinvest in. Factors like neighborhood income, demographic changes, or local competition could influence such decisions.

**Capabilities:**
An initial glance at the histograms reveals that a significant number of stores lack certain capabilities. Understanding this trend could be crucial for strategic decisions. The absence of these capabilities might be indicative of older store models, regional preferences, or specific business strategies. Delving deeper into this data could help identify opportunities for store upgrades, features based on regional needs, or even potential expansion with newer store models in untapped regions

###Scatter Plot Analysis

In [ ]:
# Scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(raw_target_data_drop3['Address.Longitude'], raw_target_data_drop3['Address.Latitude'], c='red', marker='o')
plt.title("Distribution of Target Stores")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()

### Analysis of the Geographic Distribution of Target Stores:

Upon plotting the latitude and longitude coordinates of each Target store, the scatter plot  shapes out a  silhouette resembling that of the USA. This geographic distribution provides us with several valuable insights:

**Broad Presence:** The distribution indicates that Target has a wide-reaching presence across the continental United States. The scattering of stores from coast to coast suggests a national strategy in place, aiming to cater to a diverse range of demographics and regional preferences.

**Density Variations:** While the scatter plot outlines the country, variations in the density of points indicate regions where Target has a stronger retail presence. Densely populated areas or economic hubs likely have a higher concentration of stores, signaling the company's strategic placement in locations with potentially higher foot traffic and purchasing power.

**Potential Gaps:** Conversely, areas with fewer points might represent regions with fewer Target outlets. These could either be regions that are less densely populated, or they could signify potential markets that Target has yet to tap into fully.

**Regional Strategies:** The distribution also prompts questions about regional strategies. For example, are there specific services or products that Target offers in coastal areas versus inland regions? Analyzing the store's offerings in conjunction with their locations could reveal tailored strategies for different parts of the country.

Now let's view this data as a chloropleth

In [ ]:

fig = px.scatter_mapbox(
    raw_target_data_drop3,
    lat='Address.Latitude',
    lon='Address.Longitude',
    hover_name=raw_target_data_drop3['Address.City'].astype(str) + '<br>' + \
               raw_target_data_drop3['Address.County'].astype(str),
    color_discrete_sequence=['red'],
    zoom=4, # You can adjust the zoom level
    height=300
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [ ]:
raw_target_data_drop3.head()



Having gained a comprehensive understanding of the Target dataset, and after diligently cleaning and examining each column, we are now ready to shift our focus to the Debt to Income Ratio data for further analysis.

## Debt to Income Ratio Data

Let's load in the dataset

In [ ]:
raw_debt_data = pd.read_csv('/content/drive/MyDrive/Brainstation/Target Capstone /datasets/household-debt-by-county.csv')
raw_debt_data

This dataset provides a snapshot of a specific economic metric across various counties from the first quarter of 1999. Each entry corresponds to a county, identified by its Federal Information Processing Standards (FIPS) code, such as 1001 for Autauga, Alabama. The "low" and "high" columns present a range for the debt to income ratio, offering insights into the economic variance within each county during each quarter of the year.

Upon seeing this data we can begin to think of some ways in which we might merge this dataset with the Target dataset.

In order to see this accurately we will need to merge the data not only in the right place (location), but the right time (date).

**Merging on Location**
In our household debt to income data we have a FIPS code, in order to use this we will need to convert these codes to match the Target dataset which has zipcode & county name. We will do this my using an intermediary dataset that will help us match the code to the county name & zipcode which is in the Target dataset.

**Merging on Date**
In our household debt to income data we are given Quarters rather than specfic dates. This means we will need to convert the Target dates into Quarters in order to merge.

### Hold Debt Data Inspection

Let's first clean and perform EDA on the household debt dataset

In [ ]:
raw_debt_data.head()

Looking at the data we have the date which is the first date of the quarter, the year, the quarter and the area FIPS code. We also have low and high from my research lower debt to income ratio means

**Low:** This value likely signifies the lower boundary or provides an estimate of the minimal debt carried by households within the area. When the low value is relatively small, it may imply that a portion of the population has debt levels that could be considered manageable within the scope of the dataset.

**High:** In contrast, this value is presumed to reflect the upper boundary or an estimate of the greatest amount of debt held by households in the area. When the high value is substantial, it suggests the presence of a subset of the population whose debt levels reach the uppermost range captured by the data.

Smaller amounts of household debt might suggest a lower likelihood of financial duress among households, while larger debt amounts may indicate a higher potential for financial stress. I'm curious to see how this relates to where Target stores are located.



In [ ]:
raw_debt_data.shape

In [ ]:
raw_debt_data.duplicated().sum()

In [ ]:
raw_debt_data.isnull().sum()

In [ ]:
raw_debt_data.info()

We see a considerable amount of nulls in the high column, we also see that year and quarter are of the type int64 however we will need date time column for our analysis. We will create a datetime column and then look into the nulls.


In [ ]:
# Convert 'year' and 'qtr' into datetime format
raw_debt_data['date'] = pd.to_datetime(raw_debt_data['year'].astype(str) + 'Q' + raw_debt_data['qtr'].astype(str))

Let's further investigate the nulls

In [ ]:
raw_debt_data.loc[raw_debt_data['high'].isnull()]

We see that all of the nulls in high have 3.43 in the low this leads me to believe that this ratio may represent the upper limit of household debt as reported by the Federal Reserve. We will handle the high nulls by filling them with 3.43 (the low value)

In [ ]:
raw_debt_data['high'].fillna(raw_debt_data['low'], inplace=True)

In [ ]:
raw_debt_data.isnull().sum()

### Statistical Analysis

Now that we've cleaned the data. Let's do a quick statistical analysis on the dataset

In [ ]:
statistical_summary = raw_debt_data.describe()
statistical_summary

In summary, the dataset shows a wide spread of household debt across many counties over two decades. The debt levels, as indicated by the low and high values, show considerable variability but tend to be more densely packed at the lower end of the scale. There is no indication of significant outliers in the high values, as the maximum high is equal to the maximum low, suggesting that in some counties, the range of debt is not broad. The fact that the 75th percentile of high values is less than the maximum (3.43) suggests that only a small proportion of counties have debt levels at the upper extreme.


Let's better understand the distribution of the data.

In [ ]:
numeric_debt_data = raw_debt_data.select_dtypes(include=['float64', 'int64'])


for col in numeric_debt_data.columns:
  numeric_debt_data.loc[:,col].plot(kind='hist')
  plt.title(col)
  plt.show()

Here we can see the distribution of our data I'll focus mostly on the high and low columns.

**Low** We see that we have data mostly distributed towards the middle and lower end of the scale with a peak ratio around 1.0

**High** Here we also see that the distribution mostly packed on the low end. We also see a surge around 3.4 because of the nulls that we filled.

### Trend Analysis

In [ ]:
# Trend Analysis
average_debt_over_time = raw_debt_data.groupby('date').agg({'low': 'mean', 'high': 'mean'}).reset_index()
plt.figure(figsize=(14, 7))
plt.plot(average_debt_over_time['date'], average_debt_over_time['low'], label='Average Low Debt', color = 'black')
plt.plot(average_debt_over_time['date'], average_debt_over_time['high'], label='Average High Debt',color = 'red')
plt.title('Trend of Average Household Debt Over Time')
plt.xlabel('Date')
plt.ylabel('Average Debt')
plt.legend()
plt.show()

The trend plot shows the average low and high household debt values over time. Both series appear to follow a similar trend, which is expected given that the high values were filled using the low values for missing entries.

Although over time the trends are increasing we also see some seasonality may be occuring. Let's analyze to better understand the seasonality.

In [ ]:
raw_debt_data['year'] = raw_debt_data['date'].dt.year
raw_debt_data['quarter'] = raw_debt_data['date'].dt.quarter
seasonal_debt = raw_debt_data.groupby('quarter').agg({'low': 'mean', 'high': 'mean'}).reset_index()
plt.figure(figsize=(10, 5))
plt.plot(seasonal_debt['quarter'], seasonal_debt['low'], marker='o', label='Average Low Debt')
plt.plot(seasonal_debt['quarter'], seasonal_debt['high'], marker='o', label='Average High Debt')
plt.title('Seasonal Trend of Average Household Debt')
plt.xlabel('Quarter')
plt.ylabel('Average Debt')
plt.xticks([1, 2, 3, 4], ['Q1', 'Q2', 'Q3', 'Q4'])
plt.legend()
plt.grid(True)
plt.show()

The seasonal trend plot for average household debt shows the mean low and high values for each quarter. There doesn't appear to be significant fluctuations between quarters, suggesting that there may not be strong seasonality present in this aspect of the data.

Let's now search for any outliers.

In [ ]:
# Outlier Detection
plt.figure(figsize=(14, 7))
plt.subplot(1, 2, 1)
plt.boxplot(raw_debt_data['low'], vert=False)
plt.title('Boxplot of Low Household Debt Values')
plt.subplot(1, 2, 2)
plt.boxplot(raw_debt_data['high'], vert=False)
plt.title('Boxplot of High Household Debt Values')
plt.show()

The boxplots for the low and high household debt values indicate that there are some outliers present in both distributions. We also see that high has a higher variance than low.


Now that both datasets are clean we can begin the process of merging them. At this time in the process I will save the merge for Sprint 3 this is because I am still working through the process of converting the FIPS and ZIP to County Names. However with the data I have now I am able to run models on the datasets separately.

For Sprint 3 I will merge them.



### Models

At this time the FIPS codes will be enough to be able to identify each county. Let's do a time series model to better understand county household debt over time.


### Time Series Analysis



Let's start with aggregating our data by date

In [ ]:
debt_aggregated = raw_debt_data.groupby('date').agg({'low': 'mean', 'high': 'mean'}).reset_index()

In [ ]:
time_series_low = debt_aggregated.set_index('date')['low']
time_series_high = debt_aggregated.set_index('date')['high']
time_series= time_series_low + time_series_high

Let's decompose for the low column

In [ ]:
# Decompose the time series to observe trend, seasonality, and residuals
decomposition = seasonal_decompose(time_series_low, model='additive', period=4)  # Quarterly data, hence period=4
decomposition.plot()
plt.show()

Here we see that there is seasonality found in our data and overall we see a peak in household debt increasing around 2008 and then somewhat decreasing and then plateauing for years after.

Let's decompose for high

In [ ]:
# Decompose the time series to observe trend, seasonality, and residuals
decomposition = seasonal_decompose(time_series_high, model='additive', period=4)  # Quarterly data, hence period=4
decomposition.plot()
plt.show()

Here we see the same trend an increase in household debt around 2008 and then a drop and an evening out.

For our model we will use SARIMAX which is suitable for seasonal data like quarterly debt levels. First let's do a time series for the low debt column

We'll split the data into train and test(80/20). We will model 1999 - 2018 and leave the rest for predictions. Let's start with the low column.

In [ ]:


# Group by both date and quarter, then aggregate
debt_aggregated = raw_debt_data.groupby(['date', 'quarter']).agg({'low': 'mean', 'high': 'mean'}).reset_index()

debt_aggregated

In [ ]:
# Focus on 'low' debt for the analysis
time_series = debt_aggregated.set_index('date')['low']

split_point = debt_aggregated[debt_aggregated['date'].dt.year < 2018].index.max()
# Now, let's find the index representing 80% of the data up to that split point
split_index_80 = int(split_point * 0.8)
# Split the data
train = time_series.iloc[:split_index_80]
test = time_series.iloc[split_index_80:split_point]

In [ ]:
# Fit a SARIMAX model to the training data
model = SARIMAX(train, order=(1, 0, 0), seasonal_order=(1, 1, 1, 4))
results = model.fit()

Now that we've run the model let's do some predictions.

In [ ]:
# Predict from the beginning of 2018 to the end of 2023
pred_start = '2018-01-01'
pred_end = '2023-12-31'
predictions = results.get_prediction(start=pd.to_datetime(pred_start), end=pd.to_datetime(pred_end), dynamic=True)
predicted_mean = predictions.predicted_mean
predicted_conf_int = predictions.conf_int()

Let's plot and evaluate the predictions now.

In [ ]:
import matplotlib.pyplot as plt

# Plotting the observed data, the fit, and the forecast for both training and test sets
'''
plt.figure(figsize=(12, 6))
plt.plot(train, label='Training Data', color='blue')
plt.plot(test, label='Test Data', color='orange')
plt.plot(train_predictions, label='Predicted Training Data', color='green', linestyle='--')
plt.plot(test_predictions, label='Predicted Test Data', color='red', linestyle='--')
plt.title('Time Series Forecasting: Training and Test Data with Predictions')
plt.xlabel('Date')
plt.ylabel('Debt Value')
plt.legend()
plt.show()
'''

Let's calculate the MAE and MAPE to evaluate how well our model performed.

In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error

# Assuming raw_debt_data has been loaded, preprocessed, and split into train and test sets

# Fit the SARIMAX model on the training data
model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 4))
results = model.fit()

# Predict the training set
train_predictions = results.predict(start=train.index[0], end=train.index[-1])

# Predict the test set
test_predictions = results.predict(start=test.index[0], end=test.index[-1])

# Calculate MAE and MAPE for the training set
train_mae = mean_absolute_error(train, train_predictions)
train_mape = np.mean(np.abs((train - train_predictions) / train)) * 100

# Calculate MAE and MAPE for the test set
test_mae = mean_absolute_error(test, test_predictions)
test_mape = np.mean(np.abs((test - test_predictions) / test)) * 100

# Print the results
print(f'Training MAE: {train_mae}')
print(f'Training MAPE: {train_mape}%')
print(f'Test MAE: {test_mae}')
print(f'Test MAPE: {test_mape}%')




Thes relatively low MAE on the training data indicates that the model's predictions are, on average, about 0.0483 units away from the actual values in the training set. This suggests a good fit on the training data.
Training MAPE: 3.6504671403246816%

The MAPE of approximately 3.65% on the training set implies that the model's predictions are off by 3.65% from the actual values, on average. This is a reasonably low error percentage, suggesting that the model has captured the underlying patterns in the training data quite well.
Test MAE: 0.0551989235411686

The MAE on the test data is slightly higher than on the training data but still remains low. An average deviation of 0.0552 suggests the model has managed to generalize well, though slightly less effectively than on the training data.
Test MAPE: 3.411125803650962%

The MAPE on the test set is marginally lower than on the training set, indicating a good level of prediction accuracy. A MAPE of around 3.41% is generally considered good in many applications, showing that the model's predictive capability holds up well on unseen data.

###Interpretation:
The model demonstrates a good fit and reasonable predictive accuracy, as evidenced by the low MAE and MAPE values on both the training and test sets.

The slightly higher MAE on the test set compared to the training set is normal and suggests a modest amount of overfitting to the training data. However, the similar performance on both training and test sets suggests that the model is neither overfitting nor underfitting significantly.


### Next Steps


As for next steps I will process an intermediate dataset to help me convert the FIPS code to county Names and merge the datasets together so that I will be able to analyze, compare, and predict to answer our key question: How can household debt, specifically the household debt-to-income ratio, inform Target's strategic decisions when opening new stores?


Now that we've run our base line models.My next steps are to tease out the more intricate patterns in the data, especially the non-linear relationships and interactions between features, I intend to deploy the Random Forest algorithm. I'll also use K-means clustering to categorize similar data points and employ mapping techniques for a visual representation of data patterns.I also intend to play around with the parameters to see if there is a possibility of improving the model any further.

Concluding the analysis, I'll make predictions and carve out a strategy. Using my models, I aspire to predict the most promising locations for Target stores. Based on the insights from these models, I'll forge actionable strategies, such as identifying the best regions for new stores and spotlighting areas that might undergo notable financial shifts.

#Sprint 3
For Sprint 3 I will be completing the following:
1.   Adding and cleaning additional demographic data
2.   Merging all datasets as needed
3.   Running a number of Machine Learning models
4.   Evaluating the models
5.   Comparing the models

Let's begin with adding and cleaning additional datasets. There are many ways to understand a population and many factors that might make a county a profitable or not profitable location to place a target store.

I was able to find data for the following:

1. Age by county
2. Race and Ethnicity by county
3. Personal income by county
4. Unemployment Status by county

We will clean and perform EDA on these now.

### Cleaning & EDA on Age/Sex by County Data (*)

Let's begin by reading in the population by Age and Sex Counties dataset from StatsAmerica.org

This data set contains population estimates by age and sex for the U.S., states, counties, metros, micros, and EDDs, from 2000 to 2019.

Source: U.S. Census Bureau

In [ ]:
pop_age_sex = pd.read_csv('/content/drive/MyDrive/Brainstation/Target Capstone /datasets/PopulationbyAgeandSexCounties.csv')
pop_age_sex.head(100)

Let's see if this dataset is clean by searching for nulls and duplicates.

In [ ]:
pop_age_sex.isnull().sum()

In [ ]:
pop_age_sex.duplicated().sum()

We see that we have nulls in the Population growth column. For now we will drop this column since we already have raw population growth accounted for in each year

In [ ]:
#pop_age_sex.drop('Population Growth', axis=1, inplace=True)

Let's see the data types we have in this dataset

In [ ]:
pop_age_sex.info()

As expected we see numerical data except the description fields. We will want to convert the Year column into a date time field.

Let's convert the year and then check for nulls in this dataset

In [ ]:
pop_age_sex['Year'] = pd.to_datetime(pop_age_sex['Year'], format='%Y')
pop_age_sex.head()

In [ ]:
pop_age_sex.isnull().sum()

There are no nulls in this dataset which is great! Let's check for any duplicates.

In [ ]:
pop_age_sex.duplicated().sum()

We also have no duplicates in this dataset. Let's better understand the distibution of this data.

In [ ]:
pop_age_sex.describe()

In [ ]:
pop_age_sex.groupby('Year')['Total Population'].sum().plot(kind='line')
plt.title('Total Population by Year')
plt.xlabel('Year')
plt.ylabel('Total Population')

Here we see total population over time. We see an increasing trendline with a large surge in around 2010/2011 I will do more research into why that is.

As for now lets understand the distribution of the population by age and sex group.

In [ ]:
age_columns = ['Population 0-4', 'Population 5-17',
       'Population 18-24', 'Population 25-44', 'Population 45-64',
       'Population 65+', 'Population Under 18', 'Population 18-54',
       'Population 55+']  # Add all relevant columns
pop_age_sex[age_columns].sum().plot(kind='bar', color = 'red')


We see that the majority of the population is adult between 18 - 54 and when we drill deeper we see that there are slightly more 25-44 year olds than any other group.

Let's check out the percentage change of each state's population

In [ ]:
# Calculate the percentage change in population for each state
pop_age_sex['Population Growth'] = pop_age_sex.groupby('Statefips')['Total Population'].pct_change()

# Plot
plt.figure(figsize=(15, 8))
sns.barplot(x='Statefips', y='Population Growth', data=pop_age_sex)
plt.xticks(rotation=90)
plt.title('Population Growth by State')
plt.show()


Here we see that State 15 (Hawaii) has the highest percent change in population this is pretty interesting but could be do to a smaller size in population impacting the percent change. Additions to the population might have a larger impact.

### Cleaning & EDA on Personal Income by County Data (SKIP THIS SECTION)

Now let's analyze the personal income by county data from StatsAmerica.org

BEA Personal Income, Per Capita Income
Source: U.S. Bureau of Economic Analysis

In [ ]:
personal_income = pd.read_csv('/content/drive/MyDrive/Brainstation/Target Capstone /datasets/Counties Personal Income.csv')


Let's check for nulls and duplicates

In [ ]:
personal_income.isnull().sum()

In [ ]:
personal_income.duplicated().sum()

We have no nulls or duplicates in this dataset. Let's see what data types were working with


In [ ]:
personal_income.info()

Besides the description fields we have integer data. We will need to convert the Year column to a date time field

In [ ]:
personal_income['Year'] = pd.to_datetime(personal_income['Year'], format="%Y")
personal_income.info()

Let's now analyze the data a bit more

In [ ]:
personal_income.groupby('Year')['Data'].sum().plot(kind='line')
plt.title('Data by Year')
plt.xlabel('Year')
plt.ylabel('Data')

## Cleaning & EDA on Race by County Data (*)

In [ ]:
race_data=pd.read_csv('/content/drive/MyDrive/Brainstation/Target Capstone /datasets/PopulationbyRaceCounties.csv')

In [ ]:
race_data.head()

Let's check for nulls and duplicates

In [ ]:
race_data.isnull().sum()

In [ ]:
race_data.duplicated().sum()

We see that we have a few nulls:

* White Alone                              10
* Black Alone                              10
* American Indian or Alaskan Native        10
* Asian Alone                              10
* Hawaiian or Pacific Islander Alone    31920
* Two or More Races                     31920
* Not Hispanic                             10
* Hispanic                                 10

Let's take a closer look at these nulls so that we can decide what to do with them


In [ ]:
race_data.loc[race_data['White Alone'].isnull()]

When we take a closer look we see 10 years of data for Puerto Rico is null we can assume these are what is causing the 10 nulls for.
* White Alone                              10
* Black Alone                              10
* American Indian or Alaskan Native        10
* Asian Alone                              10
* Not Hispanic                             10
* Hispanic                                 10

We will drop the Puerto Rico data since there isn't much value added in this analysis with the nulls. Let's evaluate the 31920 nulls in Pacific Islander and Two or more races.


In [ ]:
#Drop puerto rico rows

race_data = race_data[race_data['Statefips'] != 72]


In [ ]:
race_data.loc[race_data['Hawaiian or Pacific Islander Alone'].isnull()]

For this project we can assume for the Hawaiian or Pacific Islander Alone & Two or More Races there are 0 in that county. We will replace these null values with 0.

In [ ]:
race_data['Hawaiian or Pacific Islander Alone'] = race_data['Hawaiian or Pacific Islander Alone'].fillna(0)
race_data['Two or More Races'] = race_data['Two or More Races'].fillna(0)

In [ ]:
race_data.isnull().sum()

Let's do a bit of analysis on the data to better understand it.

Let's look at population over time

In [ ]:
race_data.groupby('Year')['Total Population'].sum().plot(kind='line')
plt.title('Total Population by Year')
plt.xlabel('Year')
plt.ylabel('Total Population')

As expected we see population increasing over time. Let's look at each race over time.

In [ ]:
race_columns = ['White Alone', 'Black Alone', 'American Indian or Alaskan Native', 'Asian Alone', 'Hawaiian or Pacific Islander Alone', 'Two or More Races', 'Not Hispanic', 'Hispanic']

for col in race_columns:
  race_data.groupby('Year')[col].sum().plot(kind='line')
  plt.title(col)
  plt.xlabel('Year')
  plt.ylabel(col)
  plt.show()

In general we see that the populations of each group are trending upwards as expected. In the year 1999/2000 we see a sharp decrease in population which I am unable to source the reason why. I'd like to believe it is because many people thought the world was going to end in Y2K -
[link here](https://www.eonline.com/news/541614/remember-when-everyone-thought-the-world-was-going-to-end-with-y2k) However I have no proof of this so we will just accept the decline as is.



In [ ]:
# prompt: make a bar chart of all races

race_columns = ['White Alone', 'Black Alone', 'American Indian or Alaskan Native', 'Asian Alone', 'Hawaiian or Pacific Islander Alone', 'Two or More Races', 'Not Hispanic', 'Hispanic']
race_data[race_columns].sum().plot(kind='bar', color = 'red')


## Cleaning & EDA on Unemployment by County (SKIP THIS SECTION)

This data has information from 2000-2022

Because this data is structured in a way that makes it challenging to analyze. I will not be merging this dataset to the final table for this sprint.

Please move forward to the next section "Location Data Processing"

In [ ]:
unemployment_data=pd.read_csv('/content/drive/MyDrive/Brainstation/Target Capstone /datasets/unemployment_data.csv')

In [ ]:
unemployment_data.head()

In [ ]:
unemployment_data.columns

We find that we have data from 2000-2022 which works well for our analysis. Let's check for nulls and duplicates

In [ ]:
unemployment_data.duplicated().sum()

In [ ]:
unemployment_data.isnull().sum()

There are no duplicates but we find a number of nulls. Let's explore these nulls further starting with 'Med_HH_Income_Percent_of_State_Total_2021' since it has the highest amount.

In [ ]:
unemployment_data.loc[unemployment_data['Med_HH_Income_Percent_of_State_Total_2021'].isna()]

Interestingly enough we find that there are no columns similar to HH Median Income Percent of State Total in 2021 no other year has that attribute. For this reason I will explore dropping it.

In [ ]:
unemployment_data = unemployment_data.drop(['Med_HH_Income_Percent_of_State_Total_2021'], axis=1)

Let's move on to our next null column Median Household income 2021. We find the same to be true there is no other column similar to this column so let's explore dropping it

In [ ]:
unemployment_data.loc[unemployment_data['Median_Household_Income_2021'].isna()]

In [ ]:
unemployment_data = unemployment_data.drop(['Median_Household_Income_2021'], axis=1)

In [ ]:
unemployment_data.loc[unemployment_data['Employed_2022'].isna()]

## Location Pre-processing


Let's start the process of merging the data on location (FIP, ZIP, COUNTYNAME)


. First let's read in our converter tables. It contains the county name, zipcode, state and FIPS code.

In [ ]:
zip2fip = pd.read_csv('/content/drive/MyDrive/Brainstation/Target Capstone /datasets/zip2fips.csv')
zip2fip.head()

Let's ensure the data is clean by checking for nulls and duplicates

In [ ]:
zip2fip.isnull().sum()

In [ ]:
zip2fip.duplicated().sum()

There are no duplicates or nulls.

Let's now create a column called ZIP in the Target dataset by extracting it from the Formatted Address Column, this way we will be able to merge the datasets on ZIP and County name.

In [ ]:
# extract ZIP code
raw_target_data_drop3['ZIP'] = raw_target_data_drop3['Address.FormattedAddress'].str.extract(' (\d{5})').astype(int)
raw_target_data_drop3.head()


Let's also strip the word 'County' from the county name on our converter table since the Target dataset only has the name in it ('Calhoun' vs 'Calhoun County')  

In [ ]:
zip2fip['COUNTYNAME'] = zip2fip['COUNTYNAME'].str.replace(' County', '', regex=False)
zip2fip.head()

In order to merge the columns on ZIP and County we will need to change the names of the columns so that they're matching.

In [ ]:
raw_target_data_drop3.rename(columns={'Address.County': 'COUNTYNAME'}, inplace=True)
raw_target_data_drop3.head()

##Merge Target & Household Debt to Income

Now that the data has been processed and analyzed,

Let's begin with merging the Target dataset to our first demographic dataset-- household debt to income ratio table.

In order to do so we will:
-  Merge the Target data to a converter table
-  Join the data on Quarter, Year, ZIP, and FIPS code
-  Make a new column 'LastMilestoneEvent' that has the last date something occurred at the target (opening or remodel) in order to work with the nulls in the 'Remodel' column (we want to preserve the nulls)


Let's begin by creating a copy of the cleaned dataset and we will call is 'clean_target' this way we can always refer back to the original cleaned dataset if needed.

In [ ]:
clean_target = raw_target_data_drop3.copy()

Since we have nulls that we want to perserve in the 'LocationMilestones.LastRemodelDate' column we will create a new column 'LastMilestoneEvent' that contains the latest milestone event (for any Target that has not experienced a remodel the latest event will be the open date)

In [ ]:
clean_target['LastMilestoneEvent'] = clean_target['LocationMilestones.LastRemodelDate'].fillna(clean_target['LocationMilestones.OpenDate'])
clean_target.head()

In [ ]:
clean_target.shape

Our household debt data has a column for each quarter and we'll need to merge on it so we will make a new column for quarter of the year

In [ ]:
clean_target['Quarter'] = clean_target['LastMilestoneEvent'].dt.quarter
clean_target.head()

Now let's add a Year  to join on since our household debt data also needs to be merged on 'Year'

In [ ]:
clean_target['Year'] = clean_target['LastMilestoneEvent'].dt.year
clean_target.head()

Let's check the new data types to ensure we have matching data types

In [ ]:
clean_target.info()

In [ ]:
raw_debt_data.info()

All data types needed to merge are the same (int 64)

Before we merge the data let's add a column isTarget that will help the ML model categorize locations with and without a Target store.

In [ ]:
clean_target['isTarget'] = 1
clean_target.head()

Let's check for any nulls

In [ ]:
clean_target.isnull().sum()

We see COUNTYNAME has 1 unexpected Null and Remodel has 395 expected nulls.

In [ ]:
clean_target[clean_target['COUNTYNAME'].isnull()]

We find that the City is New York. Since this is in NYC the county name will also be New York let's confirm this with other Targets located in NYC.

In [ ]:
clean_target[clean_target['Address.City'] == 'New York']

Let's fill the null county name with 'New York'

In [ ]:
clean_target['COUNTYNAME'].fillna('NY', inplace=True)


Let's also change 'Address.Subdivision' to 'State' so that we can merge.

In [ ]:
clean_target.rename(columns={'Address.Subdivision': 'STATE'}, inplace=True)


Now let's merge Target data and the converter table. We will be using an outer join because we want to preserve all of the target locations as well as any locations without a target so that we can evaluate the differences in demographics.

We will merge on the zipcode since that it is on the county level and I've noticed some inconsistencies in how the county name was written out so we will rely on zipcode.

In [ ]:
merged_data = pd.merge(clean_target, zip2fip, how='outer',on=['ZIP'])
merged_data.head(100)

In [ ]:
merged_data.shape

We see that the shape of the data has become much larger which is expected.
Let's check for Nulls, we expect to see many within the target data set since there are more counties than Target stores.

In [ ]:
merged_data.isnull().sum()

Let's dig deeper into some of the nulls that are unexpected
*  STATE                                    1
*  STCOUNTYFP                               1
*  CLASSFP                                  1





In [ ]:
merged_data[merged_data['STATE_y'].isnull()]

In [ ]:
merged_data[merged_data['STCOUNTYFP'].isnull()]

In [ ]:
merged_data[merged_data['CLASSFP'].isnull()]

We see that the nulls are all the same row of data. Because we don't want to lose any of the target data we will fill this information in.

We see that the state is WI, the county name is Waukesha, and the FIPS code is 55133. We actually won't need the ClassFP column so we can drop it.



In [ ]:
merged_data['STATE_y'].fillna('WI', inplace=True)
merged_data['STCOUNTYFP'].fillna('55133', inplace=True)
merged_data['COUNTYNAME_y'].fillna('Waukesha', inplace=True)

In [ ]:
merged_data.drop(columns=['CLASSFP'], inplace=True)

In [ ]:
merged_data.isnull().sum()

Next let's merge the House Hold Debt data to the Merged Target Data.

Let's change the name of the STCOUNTYFP to FIP inorder to merge

In [ ]:
merged_data.rename(columns={'STCOUNTYFP': 'FIP'}, inplace=True)
merged_data.head()

Let's ensure the data types are the same so that we can merge

In [ ]:
merged_data.dtypes

In [ ]:
raw_debt_data.dtypes

We see that FIP has two different data types so let's convert the Target/Merged FIPs to int64

In [ ]:

merged_data['FIP'] = merged_data['FIP'].astype('int64')


Let's change the household debt data columns to match with our other datasets so that we can merge

In [ ]:
raw_debt_data.rename(columns={'area_fips': 'FIP'}, inplace=True)
raw_debt_data.rename(columns={'year': 'Year'}, inplace=True)
raw_debt_data.rename(columns={'quarter': 'Quarter'}, inplace=True)
raw_debt_data.head()

Due to time restraints I will perform and inner join on the data

In [ ]:
merged_data_2 = pd.merge(merged_data, raw_debt_data, how='inner', on=['FIP', 'Year', 'Quarter'])
merged_data_2.head()

In [ ]:
merged_data_2.isnull().sum()

In [ ]:
merged_data_2.shape

## Merge Data by Age/Sex

Now that we have merged the Target dataset and the household debt to income ratio data. Let's add additional datasets starting with Age and sex data

In [ ]:
pop_age_sex.head(100)

Looking at the data in order to merge we will need to merge on the Year, the county, and the FIPS code. Something to note is that the household income data is per quarter while age and population is by year. I will assume that the QoQ data doesn't fluctuate too much and will be okay with the data being the same for each quarter as long as the year is aligned.

In order to merge these are the steps that we will take:

1. Pre-process fields to be able to merge (format 'Year', remove 'County' from County Name)
2. Ensure the columns are the same name
3. Merge!


Let's begin with formatting the Year column currently it is a date time format and has the date for the 1st of the year so we can easily pull the Year from the Age by County data


In [ ]:
pop_age_sex['Year'] = pop_age_sex['Year'].dt.year

In [ ]:
pop_age_sex.head(50)


Next let's remove the word 'county' from the county name and make a separate column for each state

In [ ]:
pop_age_sex['Location'] = pop_age_sex['Description'].str.replace('County', '')
pop_age_sex.head(50)

Now that we've made a new column with just the county name let's isolate the county name and state initials we'll need this for the merge.

In [ ]:
# Split the "Column" into two separate columns using the comma as the delimiter
pop_age_sex[['County', 'State']] = pop_age_sex['Location'].str.split(',', expand=True)

In [ ]:
pop_age_sex.head(50)

Let's check if there are nulls after our new column was created

In [ ]:
pop_age_sex.isnull().sum()

We now see that there are 1,052 nulls in the dataset let's explore them

In [ ]:
pop_age_sex.loc[pop_age_sex['State'].isnull()].head(50)

Looking at the nulls we see that the county column contains aggregate populations for the entire country or for the entire state for the year. We are aiming to analyze per county so we'll want to drop these rows.

In [ ]:
pop_age_sex.dropna(subset=['State'], inplace=True)

In [ ]:
pop_age_sex.isnull().sum()

Let's now drop the redundant columns

In [ ]:
pop_age_sex.drop(columns=['Description', 'Location','Statefips','Countyfips'], inplace=True)
pop_age_sex.head(50)

Now let's change the column names so that they match and we can merge. The columns we need are COUNTYNAME, FIP, STATE, YEAR

In [ ]:
pop_age_sex.rename(columns={'County': 'COUNTYNAME'}, inplace=True)
pop_age_sex.rename(columns={'State': 'STATE'}, inplace=True)
pop_age_sex.rename(columns={'IBRC_Geo_ID': 'FIP'}, inplace=True)
pop_age_sex.head(50)

In [ ]:
master_data = pd.merge(merged_data_2, pop_age_sex, how='left', on=['FIP','Year'])



In [ ]:
master_data.head(100)

Now that we have successfully merged the datasets let ensure we have no nulls

In [ ]:
master_data.isnull().sum()

We see 3 nulls in a few columns. Let's explore them further

In [ ]:
master_data.loc[master_data['COUNTYNAME_y'].isnull()].head(50)

The data will nulls are from Targets that predate our demographic data. (1988-1999) we can drop these columns since most of our demographic data starts in 2000 with some as early as 1999.

In [ ]:
master_data.dropna(subset=['COUNTYNAME_y'], inplace=True)

In [ ]:
master_data.isnull().sum()

Let's now move on to the next merge -- Personal Income.

## Merge Data by Race

In [ ]:
race_data.tail(50)

Looking at the data we will need to merge on FIPS and Year

In order to merge these are the steps that we will take:

1. Ensure the columns are the same name
2. Merge!

In [ ]:
race_data.rename(columns={'IBRC_Geo_ID': 'FIP'}, inplace=True)

Now let's merge the columns

In [ ]:
master_data = pd.merge(master_data, race_data, how='left', on=['FIP','Year'])

In [ ]:
master_data.head(50)

Let's check for any nulls that mightve occured due to the merge

In [ ]:
master_data.isnull().sum()

There are no nulls!

In [ ]:
master_data.head(50)

## Statistical Analysis on Master Dataset

Now that we have a master dataset we can perform some statistical analysis.

In [ ]:
master_data.describe()

 We will now investigate the distribution of our columns and determine which, if any, of them need to be transformed in some way.

In [ ]:
numeric_master_data = master_data.select_dtypes(include=['float64', 'int64'])


for col in numeric_master_data.columns:
  numeric_master_data.loc[:,col].plot(kind='hist')
  plt.axvline(numeric_master_data[col].mean(), label='Mean', c='red')
  plt.axvline(numeric_master_data[col].median(), label='Median', c='blue')
  plt.title(col)
  plt.show()

We could either do these logarithmic (or other) transformations of our dataset in order to remove some of the skew that we see, HOWEVER, because we will be scaling this dataset, this transformation will be mostly lost when we do scaling and PCA.

Now we will look at the correlations between our variables before we scale and transform them using dimensionality reduction techniques

In [ ]:
corr_df = master_data.corr()
mask = np.triu(corr_df)

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(corr_df, vmin=-1, vmax=1, cmap='coolwarm', annot=True, mask=mask) #annot is annotation
plt.show()

From the correlation map we see that there is some multi-colinearity.This is important to note, however, since we will be using dimensionality reduction techniques, dealing with these issues of collinearity will be done automatically

Hypothesis testing

## Scaling Master Data

For our model we will need to scale our data. Let's first filter to have only numeric columns in our dataset inorder to scale.

In [ ]:
numeric_master_data = master_data.select_dtypes(include=['float64', 'int64'])

In [ ]:
numeric_master_data.head()

Let's set our scalers a variable so that we can use them.

In [ ]:
ms = MinMaxScaler()
ss = StandardScaler()

Now let's fit the data our chosen scalars.

In [ ]:

data_minmax_scaled = ms.fit_transform(numeric_master_data)
data_standard_scaled = ss.fit_transform(numeric_master_data)

We see that these are numpy arrays

In [ ]:
type(data_minmax_scaled)

In [ ]:
type(data_standard_scaled)

Let's convert the numpy arrays into pandas dataframes so that we can work with them.

In [ ]:
df_m = pd.DataFrame(data_minmax_scaled, columns=numeric_master_data.columns, index = numeric_master_data.index)
df_s = pd.DataFrame(data_standard_scaled, columns=numeric_master_data.columns, index = numeric_master_data.index)

In [ ]:
df_m.isna().sum()

In [ ]:
np.isfinite(df_m).sum()

In [ ]:
df_m.isna().sum()

In [ ]:
df_m.isna().sum().value_counts()

## Dimensionality Reduction (SKIP THIS SECTION)

Now we want to reduce the dimensions of our datasets in order to plot the principle components against one another. We will create 6 different transformations based on 3 dimensionality reductions and the 2 scalings.

We are going to do PCA on all of the data to better understand the similarities of all targets and demographics


In [ ]:
#PCA + MIN MAX
pca_m = PCA(n_components=3)
df_m_pca = pca_m.fit_transform(df_m)

#PCA + STANDARD
pca_s = PCA(n_components=3)
df_s_pca = pca_s.fit_transform(df_s)

#TSNE + MIN MAX
tsne_m = TSNE(n_components = 3, perplexity = 15) #perplexity is a parameter
df_m_tsne = tsne_m.fit_transform(df_m)

#TSNE + STANDARD
tsne_s = TSNE(n_components = 3, perplexity = 15) #perplexity is a parameter
df_s_tsne = tsne_s.fit_transform(df_s)

#UMAP + MINMAX
umapM = UMAP(n_neighbors=10, n_components=3).fit_transform(df_m)

#UMAP + STANDARD
umapS = UMAP(n_neighbors=10, n_components=3).fit_transform(df_s)

Now that we have our 6 dimentionality reduced dataframes, it's time to look at how they are scattered. Let's start with the PCA data.

First we need to convert them from numpy data arrays to pandas dataframes.

In [ ]:
df_m_pca = pd.DataFrame(df_m_pca, index=numeric_master_data.index)
df_s_pca = pd.DataFrame(df_s_pca, index=numeric_master_data.index)

In [ ]:
#PCA MINMAX
plt.figure()
sns.pairplot(df_m_pca)
plt.show()

In [ ]:
#PCA STANDARD
plt.figure()
sns.pairplot(df_s_pca)
plt.show()

Let's now work with the TSNE data

In [ ]:
df_m_tsne = pd.DataFrame(df_m_tsne, index=numeric_master_data.index)
df_s_tsne = pd.DataFrame(df_s_tsne, index=numeric_master_data.index)

In [ ]:
#TSNE MINMAX
sns.pairplot(df_m_tsne)

In [ ]:
#TSNE STANDARD
sns.pairplot(df_s_tsne)

Now let's do the same for UMAP

In [ ]:
df_umapM = pd.DataFrame(umapM, index=numeric_master_data.index)
df_umapS = pd.DataFrame(umapS, index=numeric_master_data.index)

In [ ]:
#UMAP MINMAX
sns.pairplot(df_umapM)

In [ ]:
#UMAP STANDARD
sns.pairplot(df_umapS)

We will now look at 3d plots of these techniques to assess which is most separable

In [ ]:
#PCA Minmax

plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

ax.scatter(df_m_pca[0],df_m_pca[1],df_m_pca[2])
plt.show()

In [ ]:
#PCA Standard
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

ax.scatter(df_s_pca[2],df_s_pca[1],df_s_pca[0])
plt.show()

In [ ]:
#TSNE M


plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

ax.scatter(df_m_tsne[0],df_m_tsne[1],df_m_tsne[2])
plt.show()

In [ ]:
#TSNE S

plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

ax.scatter(df_s_tsne[2],df_s_tsne[1],df_s_tsne[0])
plt.show()


In [ ]:
#UMAP M


plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

ax.scatter(df_umapM[2],df_umapM[1],df_umapM[0])
plt.show()

In [ ]:
#UMAP S


plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')

ax.scatter(df_umapS[0],df_umapS[1],df_umapS[2])
plt.show()

## Creating a New Dataset w/ isTarget Classifier

We're going to do analysis on the just the population data and add an isTarget column we will use this as labels when clustering. We will attempt to see if there are any ripe conditions for a Target store.

First we'll make a merged data set of just the population data ( population by age and sex, household debt data, and race data)

In [ ]:
merged_demo_data = pd.merge(pop_age_sex,raw_debt_data, how='inner', on=['Year','FIP'])

In [ ]:
merged_demo2 = pd.merge(merged_demo_data,race_data, how ='inner', on=['Year','FIP'] )

In [ ]:
merged_demo2

Let's check for nulls

In [ ]:
merged_demo2.isna().sum()

No Nulls but we do have some duplicates let's drop them

Let's remove duplicates

In [ ]:
merged_demo2.drop_duplicates(inplace=True)

In [ ]:
merged_demo2

In order to run analysis we want to take a snap shot of the most recent year in our data to make predictive analysis on "Where should we open a Target?"?

In [ ]:


max_year = merged_demo2['Year'].max()
max_year


The latest year is 2019 Now let's filter only the most recent demographic data

In [ ]:
filtered_2019 = merged_demo2[merged_demo2['Year'] == 2019]
filtered_2019

Now let's extract Quarter 1 so that we have a snap shot of whats going on. We will assume data from quarter 1 is for the year

In [ ]:
annual_data = filtered_2019[filtered_2019['qtr']== 1 ]

In [ ]:
annual_data

Checking that the number of rows is the same as the number of unique values for FIP code ensures that we have no duplicate counties

In [ ]:
annual_data.nunique()

In [ ]:
master_data

Now we want to filter the FIPS codes where there is a Target
Then remove that from the dataset so that we can separate out the isTarget 0 vs 1

In [ ]:
FIPS = master_data['FIP'].unique()


In [ ]:


annual_data = annual_data[~annual_data['FIP'].isin(FIPS)]


In [ ]:
annual_data

Now that we have all of the counties where there is no Target we can add isTarget = 0

In [ ]:
annual_data['isTarget'] = 0

In [ ]:
annual_data

In [ ]:
master_data

Now let's join in historical population data where there is a target Let's update isTarget to 1 from our master dataset (contains Target & demo data)

In [ ]:
master_data['isTarget'] = 1

In [ ]:
master_data

Let's join on only the same columns (demographic data)

In [ ]:
common_columns = set(master_data.columns) & set(annual_data.columns)

common_columns

In [ ]:
result = pd.merge(master_data[common_columns],annual_data[common_columns], how = 'outer')

Now we have a dataset that contains all counties whether there is a target or not as well as a label indicating where each county falls

In [ ]:
result

Let's check for nulls from our Join

In [ ]:
result.isnull().sum()

We see that we have 5 nulls in a few columns let's take a closer look at them

In [ ]:
result[result['STATE'].isnull() == True]

There are nulls because we don't have data dating back to 1999 for all columns. Let's drop these columns.

In [ ]:


result = result.dropna(subset=['STATE'])


In [ ]:
result.isnull().sum()

## Test Run W/o Dimensionality Reduction

First let's separate X and Y and make a copy of the original dataframe/ drop redundant columns

In [ ]:
# making a copy of original dataframe and dropping problematic and redundant location columns since we have FIP codes
x = result.copy()
x = x.drop(columns=['isTarget', 'date', 'STATE', 'COUNTYNAME', 'Description','Year','FIP','Statefips','Countyfips','Quarter','qtr'])
y = result['isTarget']


In [ ]:
y

Drop date column since the model can't use it

In [ ]:
x = x.astype(float)
x.info()

First let's scale the data

In [ ]:
#min max x

x_scaled = StandardScaler().fit_transform(x)


Run a Logistic Regression

In [ ]:
#run a logistic regression

X = x_scaled
y = y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y )


# Logistic Regression
model = LogisticRegression()
model.fit(X_train, y_train)
predict_all = model.predict(X)
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


**Overall Accuracy (0.9315 or 93.15%): **This metric indicates that the model correctly predicted the class of the data points 93.15% of the time. This high accuracy suggests that the model is performing well in distinguishing between the two classes (0 and 1).

**Class-Specific Analysis:**

Class 0:
Precision (0.89 or 89%): **bold text** Out of all instances predicted as class 0, 89% actually belong to class 0. This indicates a low false positive rate for class 0.

**Recall (0.98 or 98%): **Out of all actual instances of class 0, 98% were correctly identified by the model. This indicates that the model is very effective at detecting class 0 instances.

**F1-Score (0.93):** The harmonic mean of precision and recall for class 0 is 0.93, suggesting a good balance between precision and recall for this class.\

** Class 1
Precision (0.98 or 98%)**:This high precision means that 98% of predictions made by the model for class 1 are correct, indicating a very low false positive rate.

**Recall (0.88 or 88%):** This suggests that the model correctly identifies 88% of all actual class 1 instances. While still high, it's notably lower than the recall for class 0.

**F1-Score (0.93): ** The F1-score for class 1 is also 0.93, showing a good balance between precision and recall, although the recall is somewhat lower compared to class 0.

In [ ]:
# prompt: write X to csv

x.to_csv('x_data.csv', index=False)


In [ ]:
type(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print(cm)


**Top-Left Cell (True Negatives, TN):** 574 instances were correctly predicted as not belonging to the target class (class 0).

**Top-Right Cell (False Positives, FP):** 7 instances were incorrectly predicted as belonging to the target class (class 0) when they did not.

**Bottom-Left Cell (False Negatives, FN):** 22 instances were incorrectly predicted as not belonging to the target class (class 1) when they did.

**Bottom-Right Cell (True Positives, TP):** 565 instances were correctly predicted as belonging to the target class (class 1).

Let's dive deeper into the False Positives because we can use them to better understand where market opportunities may be.

In [ ]:
#y_pred = pd.DataFrame(y_pred)
y_test = pd.DataFrame(y_test)
# X_test = pd.DataFrame(X_test)

y_test['Predictions'] = y_pred


In [ ]:
type(y_test)

In [ ]:
predict_all = pd.DataFrame(predict_all)

In [ ]:
y_test

In [ ]:
false_positives = y_test.loc[(y_test['Predictions'] == 1) & (y_test['isTarget'] == 0),:]

In [ ]:
market_opp = pd.merge(false_positives, result, how='left', left_index=True, right_index=True)

In [ ]:
market_opp

## Streamlit

In [ ]:
#streamlit app data


streamlit_data = X.copy()
streamlit_data = pd.DataFrame(streamlit_data, columns=x.columns)
streamlit_data['Predictions'] = predict_all
streamlit_data['isTarget'] = y


In [ ]:
streamlit_data['Predictions'].value_counts()

In [ ]:
streamlit_data['County'] = result['COUNTYNAME']

In [ ]:
streamlit_data

In [ ]:
streamlit_data.to_csv('streamlit_data_final2.csv', index=False)

In [ ]:
zip2fip

In [ ]:
# prompt: rename zip2fip stcountfy to FIP

zip2fip.rename(columns={'STCOUNTYFP':'FIP'}, inplace=True)


In [ ]:
market_opp2 = pd.merge(market_opp, zip2fip, how='left', on='FIP')

In [ ]:
# prompt: get the coefficients from the logistic regression

coeff = model.coef_


In [ ]:
# Now you can use
feature_names = x.columns

# Extracting coefficients
coefficients = model.coef_[0]

# Creating a DataFrame for better visualization
coefficients_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})

# Displaying the DataFrame
print(coefficients_df)

**Positive Coefficients:**

Features like 'Total Population_x', 'Quarter', 'Female Population', etc., have positive coefficients. This means that as the value of these features increases, the log-odds of the target variable being true (or the probability of the positive class) also increases.

Notably, features like 'Quarter', 'Population 18-24', 'Not Hispanic' have relatively high coefficients (above 3), indicating a stronger relationship with the target variable's likelihood.

'Total Population_x' and 'Total Population_y' have the same coefficient, suggesting they might be duplicate or highly correlated features.
Negative Coefficients:

**Negative Coefficients**

'high' and 'American Indian or Alaskan Native' have negative coefficients. An increase in these features is associated with a decrease in the likelihood of the target variable being true.

The negative coefficient for 'high' is quite small (-0.052814), implying a relatively minor influence in decreasing the likelihood compared to the impact of 'American Indian or Alaskan Native' (-0.603116).
Magnitude of Coefficients:

Features like 'Quarter', 'Population 18-24', 'Not Hispanic', and 'White Alone' have some of the largest coefficients, implying they are significant predictors in this model.

In [ ]:
# prompt: plot the coefficients

coefficients_df.sort_values('Coefficient', ascending=False).head(10)


Let's graph them

In [ ]:


# Assuming coefficients_df is your DataFrame and it has columns 'Feature' and 'Coefficient'
# Sort the DataFrame by the 'Coefficient' column in ascending order
coefficients_df_sorted = coefficients_df.sort_values(by='Coefficient')

# Now plot the sorted DataFrame
coefficients_df_sorted.plot(x='Feature', y='Coefficient', kind='bar', figsize=(10, 6))
plt.xlabel('Features')
plt.ylabel('Coefficients')
plt.title('Coefficients of Logistic Regression Model')
plt.show()


Highest negative coefficients

In [ ]:

coefficients_df.sort_values('Coefficient', ascending=True).head(5)

In [ ]:
import joblib

# Save the model to a file
joblib.dump(model, 'logistic_regression_model.pkl')


## Random Forest Classifier

In [ ]:
#run a random forest classifier

from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
modelRF = RandomForestClassifier()
modelRF.fit(X_train, y_train)
y_pred = modelRF.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:', classification_report(y_test, y_pred))


## Dimensionality Reduction

Now lets remove the isTarget column in order to cluster. We'll start with dimensionality reduction using umap and scaling our data

In [ ]:

result2 = result.drop(columns=['isTarget'])


In [ ]:
result2.info()

In [ ]:
result['isTarget'] = result['isTarget'].astype(int)

We'll want to filter for only numeric columns in order to model

In [ ]:


numeric_result = result2.select_dtypes(include=['float64', 'int64'])


Now let's scale our data. We'll use a minmax and standard scalar

In [ ]:

result_minmax_scaled = ms.fit_transform(numeric_result)
result_standard_scaled = ss.fit_transform(numeric_result)

We can't forget to turn our numpy arrays into pandas dataframes or else we won't be able to use it.

In [ ]:
results_m = pd.DataFrame(result_minmax_scaled, columns=numeric_result.columns, index = numeric_result.index)
results_s = pd.DataFrame(result_standard_scaled, columns=numeric_result.columns, index = numeric_result.index)

We'll use a series of differnt dimensionality reduction techniques to know which is the best one

In [ ]:
#UMAP + MINMAX
result_umapM = UMAP(n_neighbors=10, n_components=3).fit_transform(results_m)

#UMAP + STANDARD
result_umapS = UMAP(n_neighbors=10, n_components=3).fit_transform(results_s)

In [ ]:
df_result_umapM = pd.DataFrame(result_umapM, index=numeric_result.index)
df_result_umapS = pd.DataFrame(result_umapS, index=numeric_result.index)

In [ ]:
df_result_umapS.info()

In [ ]:
#PCA + MIN MAX
pca_m = PCA(n_components=3)
result_m_pca = pca_m.fit_transform(results_m)

#PCA + STANDARD
pca_s = PCA(n_components=3)
result_s_pca = pca_s.fit_transform(results_s)

#TSNE + MIN MAX
tsne_m = TSNE(n_components = 3, perplexity = 15) #perplexity is a parameter
result_m_tsne = tsne_m.fit_transform(results_m)

#TSNE + STANDARD
tsne_s = TSNE(n_components = 3, perplexity = 15) #perplexity is a parameter
result_s_tsne = tsne_s.fit_transform(results_s)


In [ ]:
df_result_m_pca = pd.DataFrame(result_m_pca, index=numeric_result.index)
df_result_s_pca = pd.DataFrame(result_s_pca, index=numeric_result.index)

df_result_m_tsne = pd.DataFrame(result_m_tsne, index=numeric_result.index)
df_result_s_tsne = pd.DataFrame(result_s_tsne, index=numeric_result.index)

We'll add the Target columns in for indexing

In [ ]:
df_result_umapM['isTarget'] = result['isTarget']
df_result_umapS['isTarget'] = result['isTarget']


In [ ]:
df_result_m_pca['isTarget'] = result['isTarget']
df_result_s_pca['isTarget'] = result['isTarget']

In [ ]:
df_result_m_tsne['isTarget'] = result['isTarget']
df_result_s_tsne['isTarget'] = result['isTarget']

Now we'll drop the columns so that they dont sway our clustering results

In [ ]:
###

### Identify which dataframe needs to be fed to the model from below:

In [ ]:
# drop istarget

df_result_umapM_drop = df_result_umapM.drop(columns=['isTarget'], inplace=False)
df_result_umapS_drop = df_result_umapS.drop(columns=['isTarget'], inplace=False)


In [ ]:
df_result_m_pca = df_result_umapM.drop(columns=['isTarget'], inplace=False)
df_result_s_pca = df_result_umapS.drop(columns=['isTarget'], inplace=False)

In [ ]:
df_result_m_tsne = result_m_tsne.drop(columns=['isTarget'], inplace=False)
df_result_s_tsne = result_m_tsne.drop(columns=['isTarget'], inplace=False)

In [ ]:
df_result_s_tsne.info()

Now let's take a look at our results. We'll color based on if there is a Target (isTarget column) We'll look at UMAP first

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_umapM[2],df_result_umapM[1],df_result_umapM[0], c=df_result_umapM['isTarget'])
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_umapS[2],df_result_umapS[1],df_result_umapS[0], c=df_result_umapS['isTarget'])
plt.show()

Now we'll look at PCA

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_m_pca[2],df_result_m_pca[1],df_result_m_pca[0], c=result['isTarget'])
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_s_pca[2],df_result_s_pca[1],df_result_s_pca[0], c=result['isTarget'])
plt.show()

Now we'll look at tSNE

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_m_tsne[2],df_result_m_tsne[1],df_result_m_tsne[0], c=result['isTarget'])
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_s_tsne[2],df_result_s_tsne[1],df_result_s_tsne[0], c=result['isTarget'])
plt.show()

In general we see that there is not great clustering no matter the dimensionality reduction. Let's see what we can do with a clustering algorithm K-Means.

## Clustering

Now let's run a clustering algorithm using the reduced dataset to see how the model would cluster any similarities and differences. First we'll run it using the umap minmax scaled data. We will use 2 clusters due to the fact that we want to predict if there is/should be a target or not.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


# Create a list to store the silhouette scores for each value of n_clusters
silhouette_scores_results = []

# For each value of n_clusters, fit a KMeans model and calculate the silhouette score
k2means = KMeans(n_clusters=2)
k2means.fit(df_result_umapM_drop)
cluster_labels = k2means.fit_predict(df_result_umapM_drop)
silhouette_scores_results.append(silhouette_score(df_result_umapM_drop, k2means.labels_))

# Find the value of n_clusters that maximizes the silhouette score
best_n_clusters = np.argmax(silhouette_scores_results)


# Print the silhouette score for the best value of n_clusters
print("Silhouette score for the best value of n_clusters:", silhouette_scores_results[best_n_clusters])

The silhouette score is not the best

Let's view the clusters colored by cluster labels

In [ ]:

plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_umapS[0],df_result_umapS[1],df_result_umapS[2], c= cluster_labels)
plt.show()

In [ ]:
plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.scatter(df_result_umapM[0],df_result_umapM[1],df_result_umapM[2], c= cluster_labels)
plt.show()

Here we see great separation between the two categories when using Kmeans clustering for PCA min max scaled data however not much cohesion. In the interest of time we'll use this configuration of the data to run further analysis.




In [ ]:
df_result_umapM.columns = df_result_umapM.columns.astype(str)


## Decision Tree

Let's better understand our clusters using a decision tree.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import matplotlib.pyplot as plt

cluster_labels = cluster_labels.astype(str)

df_result_umapM.columns = df_result_umapM.columns.astype(str)
#df_result_umapM_drop2 = df_result_umapM.drop('isTarget')




In [ ]:
df_result_umapM.columns.drop('isTarget')

In [ ]:

# Assuming df_result_umapM is your feature set and cluster_labels are the cluster labels
X_train, X_test, y_train, y_test = train_test_split(df_result_umapM, cluster_labels, test_size=0.25)

# Create a decision tree classifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)

# Visualize the tree
plt.figure(figsize=(20,10))
tree.plot_tree(classifier, filled=True, feature_names=df_result_umapM.columns)
plt.show()

Let's also run a decision tree using isTarget as our Y variable to better understand the results

This probably should've been done first but better late than never. Let's do a Logistic regression

## Logistic Regression

In [ ]:
# prompt: convert to a data frame
df_test = pd.DataFrame(result_minmax_scaled)
df_test.get_feature_names


Let's first try it on scaled (min max data)

In [ ]:
# Prepare the data
X = result_minmax_scaled # Features
y = result['isTarget']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))


The logistical regression works perfectly. This might mean it's over fit we can run a confusion matrix to further evaluate.

In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming y_test are the true labels and y_pred are the model's predictions
cm = confusion_matrix(y_test, y_pred)

print(cm)

**Top-Left Cell (True Negatives, TN):** 466 instances were correctly predicted as not belonging to the target class (class 0).

**Top-Right Cell (False Positives, FP):** 0 instances were incorrectly predicted as belonging to the target class (class 0) when they did not.

**Bottom-Left Cell (False Negatives, FN):** 2 instances were incorrectly predicted as not belonging to the target class (class 1) when they did.

**Bottom-Right Cell (True Positives, TP):** 467 instances were correctly predicted as belonging to the target class (class 1).

##### Key takeaways:

H**igh Sensitivity/Recall for Class 1:** The model successfully identified 467 out of 469 actual positive instances (class 1), which is nearly perfect.

**Perfect Specificity for Class 0:** All 466 actual negative instances (class 0) were correctly identified. There were no false positives, which is remarkable.

**Precision for Class 1:** Since there are no false positives, precision for class 1 is essentially perfect.

**Minimal False Negatives:**Only 2 instances that should have been classified as class 1 were missed. This is very low, suggesting the model is quite reliable in its predictions.

**Overall Accuracy:** Considering the total number of predictions (935), the model has an accuracy of [(466+467)/935] ≈ 99.79%, aligning with the high accuracy rate previously mentioned.

Let's better understand the coefficients

In [ ]:


X_train_df = pd.DataFrame(X_train)


In [ ]:
# Extracting coefficients
coefficients = model.coef_[0]

# Assuming X_train has column names, which are your feature names
feature_names = X_train_df.columns

# Creating a DataFrame for easier visualization
feature_importance = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})



# Displaying the DataFrame
print(feature_importance)


In [ ]:
print(result.columns)

feature_importance2 = pd.Series(classifier.feature_importances_, index=result.columns)
feature_importance2.nlargest(20).plot(kind='barh')

In [ ]:
X_train_df.columns

Let's also run a logistical regression on standard scaled data

In [ ]:
# Prepare the data
X2 = result_standard_scaled # Features
y2 = result['isTarget']  # Target variable

# Split the data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X2_train, y2_train)

# Make predictions and evaluate the model
y2_pred = model.predict(X2_test)
accuracy2 = accuracy_score(y2_test, y2_pred)
print("Accuracy:", accuracy2)
print("Classification Report:\n", classification_report(y2_test, y2_pred))

This works 100% of the time we will also do a confustion matrix to better understand our results.

In [ ]:
# Assuming y_test are the true labels and y_pred are the model's predictions
cm2 = confusion_matrix(y2_test, y2_pred)

print(cm2)

Same analysis as above

## Conclusion

In conclusion... always do a logistical regression first it can save you alot of work.

